# This Notebook filters the daily data to retain only one unique entry per song and add some aggregated metrics resulting in the cleaned_global19-24.csv

In [1]:
import pandas as pd
df = pd.read_csv('../data/raw_data/daily_global_charts/daily_combined/daily17-24.csv')

In [2]:
df['date'] = pd.to_datetime(df['date'])

In [4]:
df = df.sort_values('date').reset_index(drop=True)

In [12]:
df.tail()

,rank,uri,artist_names,track_name,source,peak_rank,previous_rank,days_on_chart,streams,date
584259,14,spotify:track:5G2f63n7IPVPPjfNIGih7Q,Sabrina Carpenter,Taste,Island Records,2,14,131,3704217,2024-12-31
584260,13,spotify:track:3xkHsmpQCBMytMJNiDf3Ii,Benson Boone,Beautiful Things,"Night Street Records, Inc./Warner Records Inc.",1,13,348,3771149,2024-12-31
584261,12,spotify:track:1Es7AUAhQvapIcoh3qMKDL,"The Weeknd, Playboi Carti",Timeless (feat Playboi Carti),XO / Republic Records,2,9,96,3840301,2024-12-31
584262,22,spotify:track:1YYfyJ2yCvGFCxwjsIqEK9,"Rauw Alejandro, Romeo Santos",Khé?,Sony Music Latin/Duars Entertainment,21,21,47,2950689,2024-12-31
584263,136,spotify:track:1k2pQc5i348DCHwbn5KTdc,Chappell Roan,Pink Pony Club,Chappell Roan PS/ Island,59,149,161,1603964,2024-12-31


In [13]:
"""
Group by artist and track name to avoid duplicates and do some aggregation.
"""

song_stats = df.groupby(['artist_names', 'track_name']).agg(
    peak_streams=('streams', 'max'),  
    total_streams=('streams', 'sum'),        
    max_peak_rank=('peak_rank', 'max'),   
    min_peak_rank=('peak_rank', 'min'),       
    max_days_on_chart=('days_on_chart', 'max'), 
    best_day_streams=('streams', 'max'),   
    first_appearance=('date', 'first'),
    uri=('uri', 'first'),
    source=('source', 'first'),
    count=('track_name', 'count')
).reset_index()

In [14]:
"""
Get the date of the day with the most streams for each song
"""

song_stats['best_day_date'] = song_stats.apply(
    lambda x: df[(df['artist_names'] == x['artist_names']) & 
                 (df['track_name'] == x['track_name']) & 
                 (df['streams'] == x['best_day_streams'])]['date'].values[0], axis=1
)


In [15]:
song_stats.head()

,artist_names,track_name,peak_streams,total_streams,max_peak_rank,min_peak_rank,max_days_on_chart,best_day_streams,first_appearance,uri,source,count,best_day_date
0,"$NOT, A$AP Rocky",Doja,1080441,6626024,168,97,7,1080441,2022-02-05,spotify:track:3zpGLSQ8QbbUnNjweWPLMD,300 Entertainment,7,2022-02-11
1,$uicideboy$,"...And to Those I Love, Thanks for Sticking Ar...",1056231,44508841,189,82,50,1056231,2021-09-15,spotify:track:30QR0ndUdiiMQMA9g1PGCm,G59 Records,50,2021-10-01
2,$uicideboy$,The Thin Grey Line,1523541,1523541,136,136,1,1523541,2024-06-14,spotify:track:1d966178PJg6b4kcXqoYF3,G59 Records,1,2024-06-14
3,(G)I-DLE,Nxde,1209433,33850709,105,104,31,1209433,2022-10-18,spotify:track:6NnCWIWV740gP7DQ8kqdIE,Cube Entertainment,31,2022-10-24
4,(G)I-DLE,Queencard,1555547,114709412,132,91,88,1555547,2023-05-16,spotify:track:4uOBL4DDWWVx4RhYKlPbPC,Cube Entertainment,88,2023-06-02


In [16]:
song_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9105 entries, 0 to 9104
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   artist_names       9105 non-null   object        
 1   track_name         9105 non-null   object        
 2   peak_streams       9105 non-null   int64         
 3   total_streams      9105 non-null   int64         
 4   max_peak_rank      9105 non-null   int64         
 5   min_peak_rank      9105 non-null   int64         
 6   max_days_on_chart  9105 non-null   int64         
 7   best_day_streams   9105 non-null   int64         
 8   first_appearance   9105 non-null   datetime64[ns]
 9   uri                9105 non-null   object        
 10  source             9105 non-null   object        
 11  count              9105 non-null   int64         
 12  best_day_date      9105 non-null   datetime64[ns]
dtypes: datetime64[ns](2), int64(7), object(4)
memory usage: 924.9+ 

In [17]:
unique_tracks = df['uri'].nunique()
unique_tracks

10854

In [18]:
# sort by streams 
song_stats['total_streams'] = song_stats['total_streams'].astype(int)
song_stats = song_stats.sort_values(by='total_streams', ascending=False)
song_stats.head()


,artist_names,track_name,peak_streams,total_streams,max_peak_rank,min_peak_rank,max_days_on_chart,best_day_streams,first_appearance,uri,source,count,best_day_date
8093,The Weeknd,Blinding Lights,8453567,4552239347,8,1,1858,8453567,2019-11-29,spotify:track:0sf12qNH5qcw8qpgymFOqD,Republic Records,1858,2020-03-20
3207,Harry Styles,As It Was,16103849,3640335748,40,1,1002,16103849,2022-03-31,spotify:track:4LRPiXqCikLlN15c3yImP7,Columbia,1002,2022-04-01
4553,Lewis Capaldi,Someone You Loved,3974554,3625415177,198,4,2166,3974554,2019-01-07,spotify:track:2TIlqbIneP0ZY1O0EzYLlc,Vertigo Berlin,2166,2019-10-30
2440,Ed Sheeran,Shape of You,9891056,3607629552,1,1,2407,9891056,2017-01-06,spotify:track:7qiZfU4dY1lWllzX7mPBI3,Atlantic Records UK,2407,2017-03-03
6647,"Post Malone, Swae Lee",Sunflower - Spider-Man: Into the Spider-Verse,5033261,3467020076,196,1,2187,5033261,2018-10-18,spotify:track:1A6OTy97kk0mMdm78rHsm8,Republic Records,2187,2019-01-18


In [19]:
song_stats.to_csv('cleaned_global_17-24.csv', index=False)